In [1]:
import os
os.chdir('/gpfs/home1/gkrtod35/Diffusion-TS')
os.getcwd()

'/gpfs/home1/gkrtod35/Diffusion-TS'

In [2]:
import torch
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

from engine.solver import Trainer
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader
from Utils.io_utils import load_yaml_config, instantiate_from_config
from Models.interpretable_diffusion.model_utils import normalize_to_neg_one_to_one, unnormalize_to_zero_to_one

## Orginal

전체 구간의 길이를 학습해서 만든 것이 아닌 짧게 보름치 데이터만 학습시켜 데이터를 생성한후 예측모형에 넣기 위해 진행해봄

In [3]:
df = pd.read_csv('/home1/gkrtod35/Diffusion-TS/Data/merged_data_processed_seoul.csv', low_memory=False)

In [4]:
# 15일치 데이터로 자르기
seq_len  = 360   # 학습용: 보름치 (1h 단위 → 15*24=360h)
pred_len =  24   # 테스트용: 1일치 (24h)

# 꼬리(tail)에서 잘라내기
#    – train_data: 마지막(pred_len)시간 바로 앞의 seq_len시간
#    – test_data : 마지막 pred_len시간
df_short = df[-(seq_len + pred_len) : ]  # shape (8760, C)

numeric_df_short = df_short.drop(columns=['Idx','date','time','일시'])
numeric_df_short = numeric_df_short.apply(pd.to_numeric, errors='coerce')
numeric_df_short = numeric_df_short.to_numpy()

In [5]:
# ─── 하이퍼파라미터 ───
seq_len  = 24    # 예측모형 입력 길이
pred_len = 24    # 예측모형 예측 길이
stride   = 24    # 슬라이딩 윈도우 이동 간격
win_size = seq_len + pred_len  # = 48
n_runs   = 5     # 복원 반복 횟수
feat_num = 19

# ─── 1) 원본 시계열을 윈도우(48) 단위로 잘라서 seqs 생성 ───
# data_array: 정규화(스케일링) 이전의 원본 NumPy array, shape=(T, C)
seqs = []
for i in range(0, len(numeric_df_short) - win_size + 1, stride):
    seqs.append(numeric_df_short[i : i + win_size, :])   # [48, C]
seqs = np.stack(seqs, axis=0)                       # [N_windows, 48, C]

# ─── 2) train/test 분할 & 스케일링(기존 그대로) ───
n_train      = int(len(seqs) * 0.8)
train, test  = seqs[:n_train], seqs[n_train:]

scaler = MinMaxScaler()
# train: (n_train*72, D) → fit → reshape back
train_scaled = scaler.fit_transform(train.reshape(-1, train.shape[-1]))
train_scaled = normalize_to_neg_one_to_one(train_scaled).reshape(train.shape)
# test: 같은 방식으로 transform → reshape → normalize
test_scaled = scaler.transform(test.reshape(-1, test.shape[-1]))
test_scaled = normalize_to_neg_one_to_one(test_scaled).reshape(test.shape)

# ─── 3) SeqDataset 생성 (pred_length=24) ───
class SeqDataset(torch.utils.data.Dataset):
    def __init__(self, data, pred_length=24, regular=True):
        self.data    = data
        self.regular = regular
        # 마지막 24타임스텝만 False로 마스킹
        self.mask    = np.ones_like(data, dtype=bool)
        self.mask[:, -pred_length:, :] = False  

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.data[idx]).float()
        if self.regular:
            return x
        m = torch.from_numpy(self.mask[idx]).bool()
        return x, m

train_ds = SeqDataset(train_scaled, pred_length=pred_len, regular=True)
test_ds  = SeqDataset(test_scaled,  pred_length=pred_len, regular=False)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=len(test_ds), shuffle=False)


# 모델 설정 및 학습
class Args:
    def __init__(self):
        self.config_path = './Config/solar.yaml'
        self.save_dir    = './forecasting_exp'
        self.gpu         = 0
        os.makedirs(self.save_dir, exist_ok=True)

args   = Args()
configs = load_yaml_config(args.config_path)
device  = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')

model   = instantiate_from_config(configs['model']).to(device)
trainer = Trainer(
    config=configs,
    args=args,
    model=model,
    dataloader={'dataloader': train_loader}
)
trainer.train()   # 이제 next(self.dl) 은 Tensor 하나만 반환합니다

# 반복 학습 + 복원 그리고 합치기
n_runs = 100
all_samples = []  # 여기에 매번 복원된 결과를 붙여나갑니다

for run in range(n_runs):
    print(f"=== Run {run+1}/{n_runs} ===")
    
    # 1) 학습 (config에 설정된 에포크만큼)
    trainer.train()
    
    # 2) 복원
    #    shape=[seq_len, feat_num] 은 마스킹 전 입력 시퀀스 길이입니다.
    sample_i, *_ = trainer.restore(
        test_loader,
        shape=[win_size, feat_num],
        coef=1e-2,
        stepsize=5e-2,
        sampling_steps=200
    )
    
    # 3) 역스케일 & 원래 형태로 복원
    sample_i = scaler.inverse_transform(
                   unnormalize_to_zero_to_one(sample_i.reshape(-1, feat_num))
               ).reshape(test_scaled.shape)  # (batch, seq_len, feat_num)
    
    all_samples.append(sample_i)

# 4) 하나의 큰 배열로 합치기
#    all_samples 는 길이 5짜리 리스트, 각 원소는 (batch, seq_len, feat_num) 모양
#    concatenate 하면 (5*batch, seq_len, feat_num) 크기로 합쳐집니다
merged = np.concatenate(all_samples, axis=0)

# merged.shape 출력해 보기
print("merged shape:", merged.shape)

loss: 0.177255: 100%|██████████| 1000/1000 [01:31<00:00, 10.88it/s]


training complete
=== Run 1/100 ===


loss: 0.068326: 100%|██████████| 1000/1000 [01:34<00:00, 10.58it/s]


training complete


conditional sampling loop time step: 100%|██████████| 200/200 [00:12<00:00, 16.17it/s]


=== Run 2/100 ===


loss: 0.065508:   2%|▏         | 22/1000 [00:02<01:33, 10.41it/s]


KeyboardInterrupt: 

In [13]:
import pickle
import numpy as np

with open('array.pkl', 'wb') as f:
    pickle.dump(merged, f)

In [14]:
merged.shape

(300, 48, 19)

## 시계열 길이

첫번째 셀을 실행시키고 2,3,4번째 셀 중 하나를 실행시키면 된다

데이터 학습의 길이를 30일,90일,6개월,1년 치로 자른 후 원하는 만큼을 생성 시켰습니다

병렬처리가 가능한지 보려고 run_pipeline, single_run을 만들었습니다

In [6]:
class SeqDataset(torch.utils.data.Dataset):
    def __init__(self, data, pred_length=24, regular=True):
        self.data    = data
        self.regular = regular
        # 마지막 24타임스텝만 False로 마스킹
        self.mask    = np.ones_like(data, dtype=bool)
        self.mask[:, -pred_length:, :] = False  

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = torch.from_numpy(self.data[idx]).float()
        if self.regular:
            return x
        m = torch.from_numpy(self.mask[idx]).bool()
        return x, m

# 모델 설정 및 학습
class Args:
    def __init__(self):
        self.config_path = './Config/solar.yaml'
        self.save_dir    = './forecasting_exp'
        self.gpu         = 0
        os.makedirs(self.save_dir, exist_ok=True)

args   = Args()
configs = load_yaml_config(args.config_path)
device  = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')

model   = instantiate_from_config(configs['model']).to(device)
trainer = Trainer(
    config=configs,
    args=args,
    model=model,
    dataloader={'dataloader': train_loader}
)

In [7]:
# 필요한 외부 정의 (이미 import·정의돼 있다고 가정)
#  - normalize_to_neg_one_to_one, unnormalize_to_zero_to_one
#  - SeqDataset
#  - Trainer
#  - load_yaml_config
#  - instantiate_from_config
#  - Args  (config_path·gpu 등을 포함)

import os, copy, pickle, pandas as pd
import numpy as np, torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler

# ─────────────────────────────────────────────────────────────
def run_pipeline(seq_len, pred_len, stride, n_runs, feat_num,
                 numeric_df_short, base_configs, args, save_dir):
    """
    seq_len : 학습용 입력 길이 (720, 2160, …)
    pred_len: 예측 길이 (24)
    stride  : 슬라이딩 윈도우 간격 (24 → 하루)
    n_runs  : 복원 반복 횟수
    feat_num: 변수 개수
    numeric_df_short: 원본 시계열 ndarray
    """
    # 1) 윈도우 분할
    win_size = seq_len + pred_len
    seqs = [numeric_df_short[i:i+win_size]
            for i in range(0, len(numeric_df_short)-win_size+1, stride)]
    seqs = np.stack(seqs, axis=0)                      # [N, win_size, feat_num]

    # 2) 학습/테스트 분할 + [-1,1] 스케일
    n_train = int(len(seqs)*0.8)
    train, test = seqs[:n_train], seqs[n_train:]
    scaler = MinMaxScaler()
    train_scaled = normalize_to_neg_one_to_one(
        scaler.fit_transform(train.reshape(-1, feat_num))
    ).reshape(train.shape)
    test_scaled = normalize_to_neg_one_to_one(
        scaler.transform(test.reshape(-1, feat_num))
    ).reshape(test.shape)

    # 3) Dataset / DataLoader
    train_ds = SeqDataset(train_scaled, pred_length=pred_len, regular=True)
    test_ds  = SeqDataset(test_scaled,  pred_length=pred_len, regular=False)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=len(test_ds), shuffle=False)

    # 4) YAML 복사 후 seq_length 갱신
    configs = copy.deepcopy(base_configs)
    for k in ('seq_length','seq_len','max_seq_len','max_len','context_length'):
        if k in configs['model']['params']:
            configs['model']['params'][k] = win_size
    configs['model']['params']['feature_size'] = feat_num

    # 5) 모델·트레이너 초기화
    device = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')
    model  = instantiate_from_config(configs['model']).to(device)

    # (선택) PositionalEncoding 길이 강제 교체 ─ 경로가 다르면 print(model)로 확인
    try:
        from Models.interpretable_diffusion.model_utils import LearnablePositionalEncoding
        d_model = configs['model']['params']['d_model']
        if hasattr(model, 'model') and hasattr(model.model, 'pos_enc'):
            model.model.pos_enc = LearnablePositionalEncoding(d_model, max_len=win_size).to(device)
    except ImportError:
        pass  # 모듈 경로가 다르면 생략 → seq_length 갱신만으로 충분할 수도 있음

    trainer = Trainer(config=configs, args=args,
                      model=model, dataloader={'dataloader': train_loader})

    # 6) 학습·복원 반복
    all_samples = []
    for run in range(1, n_runs+1):
        print(f"[seq_len={seq_len}] run {run}/{n_runs}")
        trainer.train()  # 내부 max_epochs 만큼 수행
        sample_i, *_ = trainer.restore(
            test_loader,
            shape=[win_size, feat_num],
            coef=1e-2, stepsize=5e-2, sampling_steps=200
        )
        sample_i = scaler.inverse_transform(
            unnormalize_to_zero_to_one(sample_i.reshape(-1, feat_num))
        ).reshape(test_scaled.shape)                    # (batch, win_size, feat_num)
        all_samples.append(sample_i)

    # 7) 합치고 저장
    merged = np.concatenate(all_samples, axis=0)
    os.makedirs(save_dir, exist_ok=True)
    out_path = os.path.join(save_dir, f"merged_seq{seq_len}.pkl")
    with open(out_path, 'wb') as f:
        pickle.dump(merged, f)
    print(f"✔ seq_len={seq_len} 완료 → {out_path}  shape={merged.shape}")

# MAIN
if __name__ == "__main__":
    # CSV 로드 & 숫자 컬럼만 추출
    df = pd.read_csv(
        '/home1/gkrtod35/Diffusion-TS/Data/merged_data_processed_seoul.csv',
        low_memory=False
    )
    df_short = df[-(8640+24):]   # 가장 최근 8640+24시간만 사용
    numeric_df_short = (df_short
                        .drop(columns=['Idx','date','time','일시'])
                        .apply(pd.to_numeric, errors='coerce')
                        .to_numpy())

    seq_len_list = [720, 2160, 4320, 8640]   # 실험할 입력 길이
    pred_len     = 24
    stride       = 24
    n_runs       = 100
    feat_num     = numeric_df_short.shape[1]

    args          = Args()
    base_configs  = load_yaml_config(args.config_path)

    for seq_len in seq_len_list:
        run_pipeline(seq_len, pred_len, stride, n_runs, feat_num,
                     numeric_df_short, base_configs, args,
                     save_dir='./merged_results')

[seq_len=720] run 1/100


loss: 54.981483:   4%|▍         | 42/1000 [00:21<08:14,  1.94it/s]


KeyboardInterrupt: 

In [8]:
# 필수 외부 정의: normalize_to_neg_one_to_one, unnormalize_to_zero_to_one,
# SeqDataset, Trainer, load_yaml_config, instantiate_from_config, Args
import os, copy, pickle, pandas as pd, numpy as np, torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler

# ─────────────────────────────────────────────────────────────
def run_pipeline(seq_len, pred_len, stride, n_runs, feat_num,
                 numeric_df_short, base_configs, args, save_dir):

    win_size = seq_len + pred_len                      # 744 등

    # 1) 윈도우 분할
    seqs = [numeric_df_short[i:i+win_size]
            for i in range(0, len(numeric_df_short)-win_size+1, stride)]
    seqs = np.stack(seqs, axis=0)

    # 2) 스케일링
    n_train = int(len(seqs)*0.8)
    train, test = seqs[:n_train], seqs[n_train:]
    scaler = MinMaxScaler()
    train_scaled = normalize_to_neg_one_to_one(
        scaler.fit_transform(train.reshape(-1, feat_num))
    ).reshape(train.shape)
    test_scaled = normalize_to_neg_one_to_one(
        scaler.transform(test.reshape(-1, feat_num))
    ).reshape(test.shape)

    # 3) DataLoader
    train_ds = SeqDataset(train_scaled, pred_length=pred_len, regular=True)
    test_ds  = SeqDataset(test_scaled,  pred_length=pred_len, regular=False)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=len(test_ds), shuffle=False)

    # 4) YAML 사본 & 길이 갱신
    configs = copy.deepcopy(base_configs)
    for k in ('seq_length','seq_len','max_seq_len','max_len','context_length'):
        if k in configs['model']['params']:
            configs['model']['params'][k] = win_size
    configs['model']['params']['feature_size'] = feat_num
    # ── 저장 끄기 ──
    configs['solver']['save_cycle'] = 10**9          # 사실상 never
    # (필요시 results_folder도 바꿀 수 있음)

    # 5) 모델·트레이너
    device = torch.device(f'cuda:{args.gpu}' if torch.cuda.is_available() else 'cpu')
    model  = instantiate_from_config(configs['model']).to(device)

    # PosEnc 강제 교체(길이 불일치 방지) ― 경로는 print(model)로 확인
    from Models.interpretable_diffusion.model_utils import LearnablePositionalEncoding
    d_model = configs['model']['params']['d_model']
    if hasattr(model, 'model') and hasattr(model.model, 'pos_enc'):
        model.model.pos_enc = LearnablePositionalEncoding(d_model, max_len=win_size).to(device)

    trainer = Trainer(config=configs, args=args,
                      model=model, dataloader={'dataloader': train_loader})
    # **checkpoint 저장 완전히 차단**
    trainer.save = lambda *a, **k: None

    # 6) 학습·복원
    all_samples = []
    for run in range(1, n_runs+1):
        print(f"[seq_len={seq_len}] run {run}/{n_runs}")
        trainer.train()
        sample_i, *_ = trainer.restore(
            test_loader,
            shape=[win_size, feat_num],
            coef=1e-2, stepsize=5e-2, sampling_steps=200
        )
        sample_i = scaler.inverse_transform(
            unnormalize_to_zero_to_one(sample_i.reshape(-1, feat_num))
        ).reshape(test_scaled.shape)
        all_samples.append(sample_i)

    merged = np.concatenate(all_samples, axis=0)
    os.makedirs(save_dir, exist_ok=True)
    out_path = os.path.join(save_dir, f"merged_seq{seq_len}.pkl")
    with open(out_path, 'wb') as f:
        pickle.dump(merged, f)
    print(f"✔ seq_len={seq_len} 완료 → {out_path}  shape={merged.shape}")


# MAIN
if __name__ == "__main__":
    df = pd.read_csv('/home1/gkrtod35/Diffusion-TS/Data/merged_data_processed_seoul.csv', low_memory=False)
    df_short = df[-(8640+24):]
    numeric_df_short = (df_short
                        .drop(columns=['Idx','date','time','일시'])
                        .apply(pd.to_numeric, errors='coerce')
                        .to_numpy())

    seq_len_list = [720, 2160, 4320, 8640]
    pred_len, stride, n_runs = 24, 24, 100
    feat_num = numeric_df_short.shape[1]

    args         = Args()
    base_configs = load_yaml_config(args.config_path)

    for seq_len in seq_len_list:
        run_pipeline(seq_len, pred_len, stride, n_runs, feat_num,
                     numeric_df_short, base_configs, args,
                     save_dir='./merged_results')


[seq_len=720] run 1/100


loss: 0.805898:  75%|███████▍  | 747/1000 [06:18<02:08,  1.97it/s] 


KeyboardInterrupt: 

In [ ]:
# 사전 정의 : normalize_to_neg_one_to_one, unnormalize_to_zero_to_one,
# SeqDataset, Trainer, load_yaml_config, instantiate_from_config, Args
import os, copy, pickle, pandas as pd, numpy as np, torch
from torch.utils.data import DataLoader
from sklearn.preprocessing import MinMaxScaler

def single_run(seq_len, gpu_id):
    pred_len, stride, n_runs = 24, 24, 100
    win_size = seq_len + pred_len

    # ========== 데이터 준비 ==========
    if 'numeric_df_short' not in globals():
        raise RuntimeError("numeric_df_short 변수부터 만드세요!")
    data = numeric_df_short               # shape (T, C)
    feat_num = data.shape[1]

    seqs = [data[i:i+win_size] for i in range(0, len(data)-win_size+1, stride)]
    seqs = np.stack(seqs, axis=0)
    n_train = int(len(seqs)*0.8)
    train, test = seqs[:n_train], seqs[n_train:]

    scaler = MinMaxScaler()
    train_scaled = normalize_to_neg_one_to_one(
        scaler.fit_transform(train.reshape(-1, feat_num))
    ).reshape(train.shape)
    test_scaled = normalize_to_neg_one_to_one(
        scaler.transform(test.reshape(-1, feat_num))
    ).reshape(test.shape)

    train_ds = SeqDataset(train_scaled, pred_length=pred_len, regular=True)
    test_ds  = SeqDataset(test_scaled,  pred_length=pred_len, regular=False)
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader  = DataLoader(test_ds,  batch_size=len(test_ds), shuffle=False)

    # ========== 모델 ==========
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    device = torch.device("cuda")

    base_cfg = load_yaml_config(Args().config_path)
    cfg = copy.deepcopy(base_cfg)
    for k in ('seq_length','seq_len','max_seq_len','max_len','context_length'):
        if k in cfg['model']['params']:
            cfg['model']['params'][k] = win_size
    cfg['model']['params']['feature_size'] = feat_num
    cfg['solver']['save_cycle'] = 10**9   # 체크포인트 끔

    model = instantiate_from_config(cfg['model']).to(device)
    # PosEnc 길이 강제
    from Models.interpretable_diffusion.model_utils import LearnablePositionalEncoding
    d_model = cfg['model']['params']['d_model']
    if hasattr(model, 'model') and hasattr(model.model, 'pos_enc'):
        model.model.pos_enc = LearnablePositionalEncoding(d_model, max_len=win_size).to(device)

    trainer = Trainer(config=cfg, args=Args(), model=model,
                      dataloader={'dataloader': train_loader})
    trainer.save = lambda *a,**k: None

    # ========== 학습/복원 ==========
    merged = []
    for r in range(1, n_runs+1):
        print(f"[GPU{gpu_id}] seq={seq_len} run {r}/{n_runs}")
        trainer.train()
        samp, *_ = trainer.restore(
            test_loader, shape=[win_size, feat_num],
            coef=1e-2, stepsize=5e-2, sampling_steps=200
        )
        samp = scaler.inverse_transform(
            unnormalize_to_zero_to_one(samp.reshape(-1, feat_num))
        ).reshape(test_scaled.shape)
        merged.append(samp)
    merged = np.concatenate(merged, axis=0)

    out_dir = './merged_results'
    os.makedirs(out_dir, exist_ok=True)
    with open(f"{out_dir}/merged_seq{seq_len}.pkl", 'wb') as f:
        pickle.dump(merged, f)
    print(f"✅ GPU{gpu_id} seq_len={seq_len} 끝  shape={merged.shape}")

# -------------------- 실제 실행 --------------------
df = pd.read_csv('/home1/gkrtod35/Diffusion-TS/Data/merged_data_processed_seoul.csv',
                 low_memory=False)
df_short = df[-(8640+24):]
numeric_df_short = (df_short
                    .drop(columns=['Idx','date','time','일시'])
                    .apply(pd.to_numeric, errors='coerce')
                    .to_numpy())

gpu_list     = list(range(torch.cuda.device_count()))  # [0,1,2,3…]
seq_len_list = [720, 2160, 4320, 8640]

for i, seq in enumerate(seq_len_list):
    gid = gpu_list[i % len(gpu_list)]
    single_run(seq, gid)            # 순차 실행

[GPU0] seq=720 run 1/100


loss: 0.203660: 100%|██████████| 1000/1000 [08:23<00:00,  1.98it/s]


training complete


conditional sampling loop time step: 100%|██████████| 200/200 [02:58<00:00,  1.12it/s]


[GPU0] seq=720 run 2/100


...

conditional sampling loop time step: 100%|██████████| 200/200 [02:58<00:00,  1.12it/s]


[GPU0] seq=720 run 12/100


loss: 0.080898: 100%|██████████| 1000/1000 [08:23<00:00,  1.99it/s]


training complete


conditional sampling loop time step:  22%|██▏       | 43/200 [01:04<03:18,  1.26s/it]

## 병렬처리
제거 하시면 됩니다.

In [ ]:
# 5) 여러 길이를 GPU별 subprocess 로 병렬 실행
import subprocess, datetime as dt, json, pathlib, sys, torch, os

seq_len_list = [4320, 8640]          # 실험할 길이
gpu_list     = list(range(torch.cuda.device_count()))
if not gpu_list:
    raise RuntimeError("CUDA GPU가 보이지 않습니다!")

# round-robin 매핑: GPU가 2장뿐이어도 자동 분배
seq_gpu = {seq: gpu_list[i % len(gpu_list)] for i, seq in enumerate(seq_len_list)}
print("실행 매핑:", json.dumps(seq_gpu, indent=2))

log_dir = pathlib.Path("logs"); log_dir.mkdir(exist_ok=True)
procs = []

for seq, gid in seq_gpu.items():
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = str(gid)      # ★ 프로세스가 자신 GPU만 보게
    log = open(log_dir / f"diff_{seq}.out", "w")

    # 현재 노트북 셀 안 코드를 그대로 실행하려면 python -c 로 호출
    # ⇒ 가장 간단: diffusion_run 을 import 후 곧바로 호출
    cmd = [
        sys.executable, "- <<END",
        "import runpy, sys, importlib, pickle, gc, torch, os;",
        "from __main__ import diffusion_run;",
        f"diffusion_run({seq}, gpu_id=0)",
        "END"
    ]
    # 위처럼 -c 블록은 문자열 인식 문제가 있으므로 방법2: 파일 저장 후 실행
    # 여기서는 runpy 방식 대신 single_run.py(앞서 작성) 경로를 호출하는 편이 안전
    cmd = [sys.executable, "single_run.py", f"--seq_len={seq}", f"--gpu={gid}"]

    print(f"[{dt.datetime.now():%H:%M:%S}] launch seq={seq} on GPU{gid}")
    procs.append(subprocess.Popen(cmd, env=env,
                                  stdout=log, stderr=subprocess.STDOUT))

# 모든 프로세스 종료 대기
for p in procs:
    p.wait()

print("★ 모든 작업 종료 (로그: logs/diff_*.out , 결과: merged_results/)")


실행 매핑: {
  "4320": 0,
  "8640": 1
}
[05:29:09] launch seq=4320 on GPU0
[05:29:09] launch seq=8640 on GPU1
★ 모든 작업 종료 (로그: logs/diff_*.out , 결과: merged_results/)


In [4]:
import pickle

# 'array.pkl'에 저장된 NumPy 배열을 불러오기
with open('/home1/gkrtod35/Diffusion-TS/merged_results/merged_seq8640.pkl', 'rb') as f:
    merged = pickle.load(f)

In [10]:
merged[10000,:,0]

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.13304804e-01,
       1.97944890e+00, 5.39680798e+00, 8.22719600e+00, 8.86804999e+00,
       8.08676004e+00, 7.47905601e+00, 5.28533205e+00, 2.41976789e+00,
       5.88211054e-01, 6.82885917e-04, 4.89600802e-02, 1.04760041e-01,
       1.07879867e-01, 7.13999403e-02, 0.00000000e+00, 0.00000000e+00,
       2.69101984e-02, 1.31736366e-02, 4.35987263e-02, 9.22769200e-02,
       9.49173275e-02, 9.59999412e-02, 4.82462504e-02, 3.32982125e-02,
       1.31101379e-01, 4.49405484e-01, 1.41408633e+00, 2.26448446e+00,
       3.69052151e+00, 5.12492549e+00, 6.85903925e+00, 6.37114927e+00,
       3.74832498e+00, 1.38520335e+00, 8.05050859e-02, 0.00000000e+00,
       1.68479104e-02, 0.00000000e+00, 0.00000000e+00, 9.68273734e-02])

In [32]:
import subprocess, os, sys

# seq_len → GPU 매핑
seq_gpu = {720:0, 2160:1, 4320:2, 8640:3}

procs = []
for seq, gid in seq_gpu.items():
    env = os.environ.copy()
    env["CUDA_VISIBLE_DEVICES"] = str(gid)          # 각 프로세스가 자기 GPU만 보게
    cmd = [sys.executable, "single_run.py",
           f"--seq_len={seq}", f"--gpu={gid}"]      # 내부에서 gpu=gid 사용
    procs.append(subprocess.Popen(cmd, env=env))

for p in procs:
    p.wait()

print("★ 모든 작업 종료")


Traceback (most recent call last):
  File "/gpfs/home1/gkrtod35/Diffusion-TS/single_run.py", line 145, in <module>
    single_run(seq_len=args.seq_len, gpu_id=args.gpu)
  File "/gpfs/home1/gkrtod35/Diffusion-TS/single_run.py", line 80, in single_run
    tr_s = normalize_to_neg_one_to_one(scaler.fit_transform(tr.reshape(-1,feat_num))).reshape(tr.shape)
  File "/home1/gkrtod35/miniconda3/envs/df/lib/python3.10/site-packages/sklearn/utils/_set_output.py", line 316, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "/home1/gkrtod35/miniconda3/envs/df/lib/python3.10/site-packages/sklearn/base.py", line 892, in fit_transform
    return self.fit(X, **fit_params).transform(X)
  File "/home1/gkrtod35/miniconda3/envs/df/lib/python3.10/site-packages/sklearn/preprocessing/_data.py", line 454, in fit
    return self.partial_fit(X, y)
  File "/home1/gkrtod35/miniconda3/envs/df/lib/python3.10/site-packages/sklearn/base.py", line 1363, in wrapper
    return fit_method(estimator, *args, 

★ 모든 작업 종료


In [41]:
# ─── 6) 최종 X_synth, Y_synth 추출 ───
X_synth = merged[:, :seq_len, :]       # (n_test, 24, C)
Y_synth = merged[:, -pred_len:, 0]     # (n_test, 24)
print(X_synth.shape)
print(Y_synth.shape)


(15, 24, 19)
(15, 24)


In [45]:
len(test_ds)

3